In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
import falkon
import torch
import numpy as np
import time

In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [51]:
data_path = "data/YearPredictionMSD.txt"
# data_path = "data/MSD_small.csv"

In [52]:
### Load the Data ###
data = pd.read_csv(data_path,header=None)
X,y = data.drop(0, axis=1), data[0]

In [53]:
#### Normalize ####
X = (X - X.mean()) / X.std()

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [55]:
sigma = 6
lam = 1e-6
m = 500

In [56]:
####### Predict with Nystrom ########
from scripts.kernel_solvers import solve_l2_problem_Nystrom
from scripts.kernel_solvers import predict_nystrom


t0 = time.time()
alpha , idx = solve_l2_problem_Nystrom(X_train.to_numpy(), y_train.to_numpy(), m=m, sigma=sigma, reg=lam)
y_pred_nys = predict_nystrom(X_test.to_numpy(), X_train.to_numpy()[idx], alpha, sigma=sigma)
t_nys = time.time() - t0

In [57]:
m = 10000

In [58]:
######## Predict with Falkon #########
X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float64)
y_train = torch.tensor(y_train.to_numpy(),dtype=torch.float64)

X_test = torch.tensor(X_test.to_numpy())
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float64)

options = falkon.FalkonOptions(keops_active="no")
kernel = falkon.kernels.GaussianKernel(sigma=sigma, opt=options)
flk = falkon.Falkon(kernel=kernel, penalty=lam, M=m, options=options)
t0 = time.time()
flk.fit(X_train,y_train)
t_falkon = time.time() - t0
y_pred_falkon = flk.predict(X_test).reshape(-1, )

In [59]:
def relative_error(y_pred, y_true):
    y_pred = torch.tensor(y_pred, dtype=torch.float64) if isinstance(y_pred, np.ndarray) else y_pred
    return torch.norm(y_pred - y_true) / torch.norm(y_true)


rel_err_nys = relative_error(y_pred_nys, y_test)
rel_err_falkon = relative_error(y_pred_falkon, y_test)

In [60]:
print(f"Nys : {rel_err_nys.item():.3f}, Time :{t_nys:.3f} \nFalkon : {rel_err_falkon.item():.3f}, Time :{t_falkon:.3f}")

Nys : 0.249, Time :4.828 
Falkon : 0.179, Time :71.895
